In [1]:
!pip uninstall spacy -y
!pip install -U spacy

Uninstalling spacy-2.2.4:
  Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 12.8MB 237kB/s 
     |████████████████████████████████| 9.1MB 48.4MB/s 
     |████████████████████████████████| 460kB 52.4MB/s 
     |████████████████████████████████| 1.1MB 48.4MB/s 
     |████████████████████████████████| 122kB 52.7MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=70b97437ecd41482f85c7505d28be524f81bd098abcd3182cdd083d7b725a8ad
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.

In [2]:
!pip install torchtext==0.6.0

     |████████████████████████████████| 71kB 5.1MB/s 
     |████████████████████████████████| 1.2MB 11.2MB/s 
  Found existing installation: torchtext 0.9.0
    Uninstalling torchtext-0.9.0:
      Successfully uninstalled torchtext-0.9.0


In [3]:
import jieba
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import pandas as pd
import spacy
import random
# from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [4]:
!mkdir ./data
!mkdir ./data/multi30k
!python -m spacy download en
!ls ./data/multi30k -al
spacy_english = spacy.load("en_core_web_sm")
!ls ./data/multi30k -al

2021-03-24 15:11:48.530810: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7MB 243kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
total 8
drwxr-xr-x 2 root root 4096 Mar 24 15:11 .
drwxr-xr-x 3 root root 4096 Mar 24 15:11 ..
total 8
drwxr-xr-x 2 root root 4096 Mar 24 15:11 .
drwxr-xr-x 3 root root 4096 Mar 24 15:11 ..


In [5]:
!python -m spacy download de
spacy_de = spacy.load("de_core_news_sm")
!ls ./data/multi30k -al

2021-03-24 15:11:57.117440: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
     |████████████████████████████████| 19.3MB 8.5MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
total 8
drwxr-xr-x 2 root root 4096 Mar 24 15:11 .
drwxr-xr-x 3 root root 4096 Mar 24 15:11 ..


In [6]:
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_english(text):
  return [token.text for token in spacy_english.tokenizer(text)]

sample run


In [7]:
sample_text = "I love machine learning"
print(tokenize_english(sample_text))

german = Field(tokenize=tokenize_de, lower=True,
               init_token="<sos>", eos_token="<eos>")

english = Field(tokenize=tokenize_english, lower=True,
               init_token="<sos>", eos_token="<eos>")

train_data, valid_data, test_data = Multi30k.splits(exts = (".en", ".de"),
                                                    fields=(english, german))

english.build_vocab(train_data, max_size=10000, min_freq=2)
german.build_vocab(train_data, max_size=10000, min_freq=2)

print(f"Unique tokens in source (en) vocabulary: {len(english.vocab)}")
print(f"Unique tokens in target (german) vocabulary: {len(german.vocab)}")

['I', 'love', 'machine', 'learning']
downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 597kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 170kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 166kB/s]


Unique tokens in source (en) vocabulary: 5893
Unique tokens in target (german) vocabulary: 7853


In [8]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

print(train_data[5].__dict__.keys())
pprint(train_data[5].__dict__.values())

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
dict_keys(['src', 'trg'])
dict_values([['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.'], ['ein', 'mann', 'in', 'grün', 'hält', 'eine', 'gitarre', ',', 'während', 'der', 'andere', 'mann', 'sein', 'hemd', 'ansieht', '.']])


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device = device)

In [10]:
count = 0
max_len_eng = []
max_len_ger = []
for data in train_data:
  max_len_eng.append(len(data.src))
  max_len_ger.append(len(data.trg))
  if count < 10 :
    print("English - ", *data.src, " Length - ", len(data.src))
    print("German - ", *data.trg, " Length - ", len(data.trg))
    print()
  count += 1

print("Maximum Length of English sentence {} and German sentence {} in the dataset".format(max(max_len_eng),max(max_len_ger)))
print("Minimum Length of English sentence {} and German sentence {} in the dataset".format(min(max_len_eng),min(max_len_ger)))

English -  two young , white males are outside near many bushes .  Length -  11
German -  zwei junge weiße männer sind im freien in der nähe vieler büsche .  Length -  13

English -  several men in hard hats are operating a giant pulley system .  Length -  12
German -  mehrere männer mit schutzhelmen bedienen ein antriebsradsystem .  Length -  8

English -  a little girl climbing into a wooden playhouse .  Length -  9
German -  ein kleines mädchen klettert in ein spielhaus aus holz .  Length -  10

English -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15
German -  ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster .  Length -  15

English -  two men are at the stove preparing food .  Length -  9
German -  zwei männer stehen am herd und bereiten essen zu .  Length -  10

English -  a man in green holds a guitar while the other man observes his shirt .  Length -  15
German -  ein mann in grün hält eine gitarre , während der andere

iterator dim = [max_src_len, batch_size]

In [11]:
count = 0
for data in train_iterator:
  if count < 1 :
    print("Shapes", data.src.shape, data.trg.shape)
    print()
    print("English - ",*data.src, " Length - ", len(data.src))
    print()
    print("German - ",*data.trg, " Length - ", len(data.trg))
    temp_eng = data.src
    temp_ger = data.trg
    count += 1

Shapes torch.Size([12, 32]) torch.Size([18, 32])

English -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([   4,    4,  697,   46,  342,    7,    4,    4, 2322,   16,   30,    4,
         176,    4,    4,    4,    4,   16,    4,    7,  322,    4,    4,    4,
           4,    4,   48,    4,    4,   64,    4,    4], device='cuda:0') tensor([  24,   14,    6, 1094,   12,    9,   14,   38,   17,   63,    6,   33,
          17,   24,  295,    9,  267, 1397, 5610,   30,   15,   24,   64,    9,
          38,   14,   19,    9,    9,  169,    9,    9], device='cuda:0') tensor([  33,    6, 1077, 1846,    4,  241,    6,   12,  399,   22,   31,    6,
         202,   33,  467,    6,   39,   30,  962,   17,  322,   33,    6,   73,
          12,   22,   17,   10,    6,    4,  140,   22], device='cuda:0') tensor([  10,  204,   32,  601,   39,   27,   52,   19,   66,  175, 4695,    4,
          19,   22,   89,  

取得numpy型式的值


In [12]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_ger_idx = (temp_ger).cpu().detach().numpy()

In [13]:
df_eng_idx = pd.DataFrame(
    data = temp_eng_idx, columns = [str("S_")+str(x) for x in range(1, 33)])
df_eng_idx.index.name = 'Time Steps'
df_eng_idx.index = df_eng_idx.index + 1 
# df_eng_idx.to_csv('/content/idx.csv')
df_eng_idx

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,4,4,697,46,342,7,4,4,2322,16,30,4,176,4,4,4,4,16,4,7,322,4,4,4,4,4,48,4,4,64,4,4
3,24,14,6,1094,12,9,14,38,17,63,6,33,17,24,295,9,267,1397,5610,30,15,24,64,9,38,14,19,9,9,169,9,9
4,33,6,1077,1846,4,241,6,12,399,22,31,6,202,33,467,6,39,30,962,17,322,33,6,73,12,22,17,10,6,4,140,22
5,10,204,32,601,39,27,52,19,66,175,4695,4,19,22,89,146,13,17,8,195,3920,22,4,340,938,26,37,230,4,319,4,4
6,917,2030,775,336,6,157,1391,100,494,0,17,31,57,4,8,107,1822,78,44,0,722,4,1070,2766,152,11,6,51,231,99,153,29
7,4,212,13,82,3339,51,60,57,8,47,269,23,12,590,109,20,292,6,2771,18,1567,430,10,4565,40,25,95,4,10,838,189,23
8,715,171,19,57,2205,7,4,20,7,20,6,10,4,6,12,27,66,4,6,663,544,67,8,102,4,483,222,262,483,6,10,10
9,2854,4,6,7,12,291,39,305,52,167,4,57,59,21,27,1548,710,1913,7,7,7,10,4,1800,101,4,101,59,4,4,45,37


In [14]:
idx_2_word = {}

for idx, w in enumerate(english.vocab.itos):
  idx_2_word[idx] = w

df_eng_word = pd.DataFrame(columns = [str("S_")+str(x) for x in range(1, 33)])
df_eng_word = df_eng_idx.replace(idx_2_word)
# df_eng_word.to_csv('/content/Words.csv')
df_eng_word

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>
2,a,a,artist,one,photo,the,a,a,jockeys,two,men,a,there,a,a,a,a,two,a,the,run,a,a,a,a,a,three,a,a,person,a,a
3,young,woman,in,professional,of,man,woman,group,are,children,in,girl,are,young,motorcycle,man,busy,uniformed,showgirl,men,",",young,person,man,group,woman,people,man,man,doing,man,man
4,girl,in,courtyard,wrestler,a,rides,in,of,racing,wearing,red,in,many,girl,rider,in,street,men,puts,are,run,girl,in,dressed,of,wearing,are,is,in,a,holds,wearing
5,is,purple,sitting,leaps,street,his,green,people,their,jeans,bowties,a,people,wearing,stands,glasses,with,are,on,using,careful,wearing,a,like,pedestrians,black,playing,climbing,a,trick,a,a
6,rolling,leggings,drawing,onto,in,bicycle,skates,stand,horses,<unk>,are,red,outside,a,on,looks,shoppers,riding,her,<unk>,do,a,wheelchair,late,walk,and,in,up,uniform,bike,baby,blue
7,a,runs,with,another,chinatown,up,through,outside,on,water,singing,shirt,of,bikini,top,at,making,in,lipstick,to,n't,party,is,80,down,white,snow,a,is,move,who,shirt
8,giant,across,people,outside,section,the,a,at,the,at,in,is,a,in,of,his,their,a,in,break,hit,hat,on,'s,a,crossing,covered,very,crossing,in,is,is
9,snowball,a,in,the,of,brick,street,night,green,each,a,outside,large,an,his,electronic,way,motorized,the,the,the,is,a,movie,city,a,city,large,a,a,holding,playing


* LSTM輸入格式需為[src_len, batch_size, feature_size](batch_first – 默認是False)
* 以我們的例子，需帶入[src_len, 32, 300]進入LSTM
* LSTM輸出有:
  * outputs: [src_len, 32, num_directions * hidden_size]
  * hidden_state: [num_layers * num_directions, 32, hidden_size]
  * cell_state: [num_layers * num_directions, 32, hidden_size]

In [15]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()
    """
    input_size: encoder的輸入，one hot vector 的型式，也就是字典大小
    embedding_size: embedding dim的大小，表示你希望把字濃縮成幾維的向量
    hidden_size: 賦予lstm 隱藏層的神經元數量
    num_layers: 想要幾層的lstm
    p: dropout的%數
    """
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.dropout = nn.Dropout(p)
    # 我們的例子:[5893, 300]
    self.embedding = nn.Embedding(input_size, embedding_size)
    # [300, 2, 1024]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  def forward(self, x):
    """
    x: 我們上一個cell包起來的樣子[每一個batch的max_src_len, batch_szie]
    """
    # embedding: [26, 32, 300]
    embedding = self.dropout(self.embedding(x))

    # outputs [26, 32, 1024] [max_src_len , batch_size , hidden_size]
    # (hs, cs) [2, 32, 1024] , [2, 32, 1024] [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)
    
    return hidden_state, cell_state

In [16]:
input_size_encoder = len(english.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(5893, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [17]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()
    """
    input_size: decoder的輸入，one hot vector 的型式，也就是字典大小，但這次是德文的decoder
    embedding_size: embedding dim的大小，表示你希望把字濃縮成幾維的向量
    hidden_size: 賦予lstm 隱藏層的神經元數量
    num_layers: 想要幾層的lstm
    p: dropout的%數
    output_size: 最後輸出的長度，也是會表示成字典大小，看哪個德文的詞出現的機率大，這裡input_size = output_size
    """
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.output_size = output_size
    self.dropout = nn.Dropout(p)
    # 我們的例子:[7853, 300]
    self.embedding = nn.Embedding(input_size, embedding_size)
    # [300, 2, 1024]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
    # [1024, 7853] 
    self.fc = nn.Linear(hidden_size, output_size)
  
  def forward(self, x, hidden_state, cell_state):
    """
    x: [batch_size]
    hidden_state: 由encoder傳進來 [26, 32, 1024]
    cell_state：由encoder傳進來 [2, 32, 1024]
    """
    # [1, 32]
    x = x.unsqueeze(0)

    # [1, 32, 300] [1, batch_size, embedding dims(對應nn.LSTM的input_size)]
    embedding = self.dropout(self.embedding(x))

    # outputs [1, 32, 1024] [1, batch_size , hidden_size]
    # (hs, cs) [2, 32, 1024] , [2, 32, 1024] [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # [1, 32, 7853] [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # [32, 7853] [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state


input_size_decoder = len(german.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(german.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(7853, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=7853, bias=True)
)


In [18]:
for batch in train_iterator:
  print(batch.src.shape)
  print(batch.trg.shape)
  break

x = batch.trg[1]
print(x)

torch.Size([11, 32])
torch.Size([13, 32])
tensor([   5,    5,  168, 5561,    8,    5,    5,    8,    8,  105,    8,   76,
          18,    5,    5,    8,    5,   18,   18,   39,   36,    5,   18,    8,
          18,   18,    5,   18,    5,   18,    8,   43], device='cuda:0')


In [19]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM
  
  def forward(self, source, target, tfr = 0.5):
    """
    source: English [src_len, batch_size]
    target: german [src_len, batch_size]
    tfr: teacher force ratio
    """
    batch_size = source.shape[1]
    target_len = target.shape[0]
    target_vocab_size = len(german.vocab)

    # [target_len, 32, 7853]
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # [2, 32, 1024]
    hidden_state, cell_state = self.Encoder_LSTM(source)
    
    # token <SOS> [32] 只有batch_size的長度
    x = target[0]

    for i in range(1, target_len):
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output

      # 0th dimension is batch size, 1st dimension is word vocab size
      best_guess = output.argmax(1)

      # 若使用teacher forcing 就是用原始的input(german)
      x = target[i] if random.random() < tfr else best_guess
    

    return outputs

In [20]:
# Hyperparameters

learning_rate = 0.001
writer = SummaryWriter(f"runs/loss_plot")
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [21]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(5893, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(7853, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=7853, bias=True)
  )
)

In [22]:
def translate_sentence(model, sentence, german, english, device, max_length=50):
    """
    model: seq2seq
    sentence: English
    german: torchtext field
    english: torchtext field
    """
    spacy_eng = spacy.load("en_core_web_sm")

    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_eng(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    # 補足sos
    tokens.insert(0, english.init_token)
    # 補足eos
    tokens.append(english.eos_token)
    # 轉成id
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    # 轉成tensor， 並變成[src_len, 1]，符合encoder格式
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state，不做梯度
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [german.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == german.vocab.stoi["<eos>"]:
            break

    translated_sentence = [german.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

# 用來評估模型的函式: bleu
def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, './checkpoint-NMT')
    torch.save(model.state_dict(),'./checkpoint-NMT-SD')

In [23]:
epoch_loss = 0.0
num_epochs = 100
best_loss = 999999
best_epoch = -1
sentence1 = "two young white males are outside near many bushes"
ts1  = []

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  translated_sentence1 = translate_sentence(model, sentence1, german, english, device, max_length=50)
  print(f"Translated example sentence 1: \n {translated_sentence1}")
  ts1.append(translated_sentence1)

  model.train(True)
  for batch_idx, batch in enumerate(train_iterator):
    input = batch.src.to(device)
    target = batch.trg.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()
    step += 1
    epoch_loss += loss.item()
    writer.add_scalar("Training loss", loss, global_step=step)

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss / len(train_iterator))


Epoch - 1 / 100
Translated example sentence 1: 
 ['r', 'stöcke', 'sicherheitsausrüstung', 'künstlerischen', 'künstlerischen', 'künstlerischen', 'klub', 'welpen', 'welpen', 'schutzweste', 'ernte', 'weinflasche', 'weißem', 'sonnenschirm', 'sonnenschirm', 'sonnenschirm', 'sonnenschirm', 'euro', 'tennisspielerin', 'lauscht', 'lauscht', 'lauscht', 'armut', 'armut', 'gepolsterter', 'spiegel', 'tennisspielerin', 'schleudert', 'vorlesung', 'sonnenschirm', 'sonnenschirm', 'sonnenschirm', 'sonnenschirm', 'zustand', 'künstlerischen', 'künstlerischen', 'künstlerischen', 'klub', 'klub', 'welpen', 'welpen', 'jim', 'markisen', 'au', 'eyeliner', 'stapel', 'au', 'zustand', 'signiert', 'teddybären']
saving

Epoch_Loss - 4.366153240203857

Epoch - 2 / 100
Translated example sentence 1: 
 ['zwei', 'junge', 'männer', 'spielen', 'auf', '.', '<eos>']
Epoch_Loss - 2.815384864807129

Epoch - 3 / 100
Translated example sentence 1: 
 ['zwei', 'junge', 'männer', 'männer', 'in', '<unk>', '.', '<eos>']
Epoch_Loss -